
<h1><img src="https://simonsanvil.github.io/docling-workshop-pycones-2025/images/DoclingDuck.png" alt="Docling Logo" style="height: 1.5em; vertical-align: middle; margin-right: 0.5em;" />Transforma tus documentos en datos listos para IA con Docling</h1>


## Lab 1: Conversión de Documentos

Bienvenido a el primer lab de esta workshop de Docling. Este viaje de tres partes te llevará desde los conceptos básicos del procesamiento de documentos hasta la construcción de sistemas de IA avanzados y transparentes.

## Objetivos

En este primer lab, aprenderás a transformar documentos complejos en datos estructurados listos para IA. Pero esto no se trata solo de extracción, sino de una conversión inteligente que preserva todo lo importante para las aplicaciones de IA posteriores.

### Al final de este lab, habrás dominado:

- **Carga de Documentos**: Ingesta de PDFs, documentos de Word, PowerPoints y más
- **Extracción de Estructura**: Preservando de la jerarquía y las relaciones entre los elementos
- **Excelencia en Tablas**: Conversión de tablas complejas en formatos utilizables.
- **Manejo de Imágenes**: Extracción y preparación de imágenes para el procesamiento de IA.
- **Preservación de Metadatos**: Mantenimiento de la información necesaria para referenciar y contextualizar los datos.

---

## Introducción

**[Docling](https://docling-project.github.io/docling/)** es una herramienta de código abierto para el procesamiento, análisis y conversión de documentos diseñado para aplicaciones de IA generativa.

### Características Clave
- **Soporte multi-formato**: PDF, DOCX, XLSX, HTML, imágenes y más
- **Comprensión avanzada de PDF**: Diseño de página, orden de lectura, estructura de tabla, bloques de código, fórmulas
- **Representación unificada de DoclingDocument**: Estructura de datos consistente en todos los formatos
- **Opciones de exportación flexibles**: Markdown, HTML, JSON, DocTags
- **Ejecución local**: Permite procesar datos sensibles sin servicios externos
- **Integración con frameworks**: LangChain, LlamaIndex y otros frameworks de IA

## ¿Por qué usamos Docling para la conversión de documentos?

Los datos son la base de todos los sistemas de IA. Para aprovechar la mayor cantidad de datos posible, necesitamos poder ingerir datos de varios formatos con precisión. Sin embargo, los LLM generalmente requieren datos en un formato específico, de ahí la necesidad de conversión.

**Sin una conversión adecuada**:
- La información se pierde o se desordena
- Las tablas se convierten en texto ilegible
- Las imágenes desaparecen por completo
- La estructura del documento se destruye

**Con la conversión avanzada de Docling**:
- Cada pieza de información se preserva
- Las tablas mantienen su estructura
- Las imágenes se extraen y son procesables
- El diseño y las relaciones se comprenden

### Instalación Básica

In [ ]:
!uv pip install docling matplotlib pillow pandas python-dotenv

### Importamos los componentes esenciales de la librería


In [ ]:
from pathlib import Path

# Core Docling imports
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import PdfFormatOption

# For advanced features
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem, TextItem, DoclingDocument

# For data processing and visualization
import matplotlib.pyplot as plt

# Create output directory
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

## 1. Conversión Básica de Documentos

### Ejemplo Mínimo

La forma más sencilla de convertir un documento es inicializar un `DocumentConverter` y llamar a su método `convert()` con la ruta del fichero o ficheros a convertir.


In [ ]:
# Docling Technical Report
docling_paper = "https://arxiv.org/pdf/2501.17887"

In [ ]:
# Conversión simple

# Crear una instancia del convertidor con parametros por defecto
converter = DocumentConverter()

# Convertir un documento
result = converter.convert(docling_paper)
doc = result.document

In [ ]:
# Exportar a Markdown
md_out = doc.export_to_markdown()

# Imprimir un extracto del resultado
print(f"{md_out[:2000]}...")

### Exploración de la Estructura del Documento

Uno de las superpoderes de Docling es comprender la estructura del documento, algo crítico a la hora de hacer chunking inteligente o extraer partes específicas del documento.

In [ ]:
# Document metadata
print(f"Titulo: {doc.name}")
print(f"Numero de paginas: {len(doc.pages)}")
print(f"Numero de tablas: {len(doc.tables)}")
print(f"Numero de imagenes: {len(doc.pictures)}")

# Exploramos la estructura del documento
print("\nEstructura del documento:")
for i, (item, level) in enumerate(doc.iterate_items()):
    if i < 10:  # Show first 10 items
        item_type = type(item).__name__
        text_preview = item.text[:200] if hasattr(item, 'text') else 'N/A'
        print(f"{'  ' * level}{item_type}: {text_preview}")
    else:
        break

### Opciones de Exportación

Docling ofrece varias opciones de exportación para adaptarse a diferentes necesidades.

In [ ]:
markdown_text = doc.export_to_markdown()  # exportar a markdown (string)
html_text = doc.export_to_html() # exportar a html (string)
json_dict = doc.export_to_dict() # exportar a dict (json serializable)
doc_tags = doc.export_to_doctags() # exportar a doctags (string)

# Guardar el documento en varios formatos
doc.save_as_markdown(
    output_dir / "document.md",
    image_mode=ImageRefMode.PLACEHOLDER,
    image_placeholder="<!-- my image placeholder -->",
    # ...
)

# ...

# Exporta a JSON
doc.save_as_json(
    output_dir / "document.json",
)

Siéntete libre de explorar los distintos formatos de exportación y visualizar los resultados en comparación con el PDF original.

## 2. Inspecciona la salida de DoclingDocument


### Tratar con Tablas

Docling ofrece excelentes capacidades de extracción de tablas gracias a los modelos [TableFormer](https://github.com/docling-project/docling-ibm-models) que impulsan la extracción de tablas. Para este ejemplo, utilicemos un documento con más tablas.

In [ ]:
tables_example = "https://arxiv.org/pdf/2206.01062"

In [ ]:
# Convertimos el documento
table_result = converter.convert(tables_example)
table_doc = table_result.document

In [ ]:
print(f"El documento contiene {len(table_doc.tables)} tablas")

# Exportar todas las tablas
for table_idx, table in enumerate(table_doc.tables):
    # Convertir a pandas DataFrame
    df = table.export_to_dataframe(doc=table_doc)

    print(f"\n## Tabla {table_idx}")
    print(f"Dimensiones: {df.shape}")
    display(df.head())

    # Save as CSV
    df.to_csv(output_dir / f"table_{table_idx}.csv", index=False)

    # Save as HTML
    with open(output_dir / f"table_{table_idx}.html", "w") as fp:
        fp.write(table.export_to_html(doc=table_doc))

### Extracción de Imágenes

Tal como las tablas, Docling también extrae imágenes y las hace accesibles en la estructura del documento.

#### Extracción y Visualización de Imágenes

Podemos configurar la pipeline de Docling para generar imágenes de las páginas del PDF y extraer las imágenes embebidas en el documento. Esto es especialmente útil para documentos con gráficos, diagramas, fotos u otros elementos visuales.

In [ ]:
IMAGE_RESOLUTION_SCALE = 2.0  # 2x resolution (144 DPI)

pipeline_options = PdfPipelineOptions(
    images_scale=IMAGE_RESOLUTION_SCALE,
    generate_page_images=True,
    generate_picture_images=True,
)

converter_with_images = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

# Conversión con extracción de imágenes
img_result = converter_with_images.convert(docling_paper) # utilizamos el paper de docling como ejemplo
img_doc = img_result.document

In [ ]:
# Crear directorio para imágenes
images_dir = output_dir / "images"
images_dir.mkdir(exist_ok=True)

# Guardar imágenes de las páginas
for page_no, page in img_doc.pages.items():
    page_image_filename = images_dir / f"page_{page_no}.png"
    with page_image_filename.open("wb") as fp:
        page.image.pil_image.save(fp, format="PNG")

print(f"Saved {len(img_doc.pages)} page images")

Revisa la carpeta `output/` para ver las imágenes extraídas.

### Extraer y Guardar Tablas e Imágenes

Para un procesamiento personalizado, también podemos extraer figuras y tablas como imágenes:

In [ ]:
# Extraer y guardar tablas e imágenes
table_counter = 0
picture_counter = 0

for element, level in img_doc.iterate_items():
    if isinstance(element, TableItem):
        table_counter += 1
        image_filename = images_dir / f"table_{table_counter}.png"
        with image_filename.open("wb") as fp:
            element.get_image(img_doc).save(fp, "PNG")

    elif isinstance(element, PictureItem):
        picture_counter += 1
        image_filename = images_dir / f"figure_{picture_counter}.png"
        with image_filename.open("wb") as fp:
            element.get_image(img_doc).save(fp, "PNG")

print(f"Extraído {table_counter} tabla(s) y {picture_counter} figuras como imágenes")

### Inspeccionar el Contenido de las Imágenes

Docling preservará automáticamente los subtítulos y extraerá el contenido de texto de las imágenes extraídas. Veamos qué se ha extraído:

In [ ]:
def inspect_pictures_with_images(doc: DoclingDocument, image_size=(6, 4)):
    """Display pictures inline with their text content"""
    for idx, picture in enumerate(doc.pictures):
        print(f"\n{'='*60}")
        print(f"Imagen {idx}")
        print(f"{'='*60}")

        # Display the image
        try:
            img = picture.get_image(doc)
            if img:
                plt.figure(figsize=image_size)
                plt.imshow(img)
                plt.axis('off')
                plt.title(f"Picture {idx}")
                plt.show()
        except Exception as e:
            print(f"Could not display image: {e}")

        # Display metadata
        caption = picture.caption_text(doc)
        if caption:
            print(f"Subtitulo: {caption}")

        if hasattr(picture, 'prov') and picture.prov:
            print(f"Location: Page {picture.prov[0].page_no}")

        # Display embedded text
        print("\nTexto embebido:")
        text_found = False
        for item, level in doc.iterate_items(root=picture, traverse_pictures=True):
            if isinstance(item, TextItem):
                print(f"{'  ' * (level + 1)}- {item.text}")
                text_found = True

        if not text_found:
            print("  (No text elements found)")

# Use the simple inline display
inspect_pictures_with_images(img_doc)

### Visualización de la Estructura del Documento con Bounding Boxes

Para entender cómo se extrae cada parte del documento, visualicemos los elementos extraídos. Podemos hacer esto utilizando uno de los visualizadores integrados en Docling:

In [ ]:
from docling_core.transforms.visualizer.layout_visualizer import LayoutVisualizer

layout_visualizer = LayoutVisualizer()
page_images = layout_visualizer.get_visualization(doc=img_doc)

num_pages_to_viz = 2  # first N pages to visualize
pages_to_viz = list(page_images.keys())[:num_pages_to_viz]
for page in pages_to_viz:
    display(page_images[page])

## 3. Enriquecimiento de Imagenes con VLM

### Clasificación y Descripción de Imágenes

Además de las capacidades básicas de subtitulado, Docling también puede procesar imágenes utilizando LLMs multimodales. Esto nos dará descripciones de imágenes más detalladas para ayudarnos a aprovechar los datos de las imágenes de manera más efectiva.

Existen dos formas de hacer esto en Docling:

1. Con modelos de HuggingFace cargados localmente: Este método es gratuito pero generalmente requiere hardware potente o utilizar modelos pequeños como [SmolVLM](https://huggingface.co/HuggingFaceTB/SmolVLM-500M-Instruct) que pueden no ser tan precisos.
2. Utilizando un VLLM basado en API: Si estas sirviendo un modelo de visión a través de una API como Ollama, [watsonx.ai](https://watsonx.ai/), o cualquiera que sea compatible con la API de OpenAI, puedes usarla para procesar las imágenes. Este método es generalmente más preciso y fácil de usar, pero puede incurrir en costos dependiendo del proveedor y el uso.

In [ ]:
RUN_LOCAL_VLM = True

In [ ]:
from docling.datamodel.pipeline_options import PictureDescriptionApiOptions, PictureDescriptionVlmOptions

# Configure enrichment pipeline
if RUN_LOCAL_VLM:
    # activamos enriquecimiento de imagenes con un modelo VLM local
    # (NOTA: si tu runtime no tiene GPUs, el rendimiento puede llegar a ser muy lento)
    VLM_REPO_NAME = "HuggingFaceTB/SmolVLM-500M-Instruct"
    VLM_MODEL_NAME = VLM_REPO_NAME.split("/")[-1]
    enrichment_options = PdfPipelineOptions(
        do_picture_description=True,
        picture_description_options=PictureDescriptionVlmOptions(
            repo_id=VLM_REPO_NAME,
            prompt="Describe in detail what is depicted in the image",
            generation_config=dict(
                max_new_tokens=400,
                do_sample=False
            )
        ),
        generate_picture_images=True, # preserva las imagenes de las figuras para poder exportarlas luego
        images_scale=1.0,
    )
else:
    # activamos enriquecimiento de imagenes con un modelo VLM remoto (requiere endpoint compatible con OpenAI)
    VLM_MODEL_NAME = "granite-vision-3.3-2b"
    enrichment_options = PdfPipelineOptions(
        do_picture_description=True,
        enable_remote_services=True,
        picture_description_options=PictureDescriptionApiOptions(
            # cualquier endpoint de LLMs compatible con la API de OpenAI funcionaría (e.g., ollama, litellm, LMStudio, ...)
            url="http://0.0.0.0:4000/v1/chat/completions",  
            params=dict(
                model=VLM_MODEL_NAME,
                seed=42,
                max_tokens=400,
                provenance=VLM_MODEL_NAME,
            ),
            prompt="Give a detailed description of what is depicted in the image",
            concurrency=4, 
            picture_area_threshold=0.1,
            timeout=90,
        ),
        generate_picture_images=True, # preserva las imagenes de las figuras para poder exportarlas luego
        images_scale=1.0,
    )


converter_enriched = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=enrichment_options)
    }
)
enr_result = converter_enriched.convert(docling_paper)
enr_doc = enr_result.document

In [ ]:
from docling_core.types.doc.document import PictureDescriptionData
from IPython import display

html_buffer = []
# display the first 5 pictures and their captions and annotations:
for pic in enr_doc.pictures[:5]:
    html_item = (
        f"<h3>Picture <code>{pic.self_ref}</code></h3>"
        f'<img src="{pic.image.uri!s}" /><br />'
        f"<h4>Caption</h4>{pic.caption_text(doc=doc)}<br />"
    )
    for annotation in pic.annotations:
        if not isinstance(annotation, PictureDescriptionData):
            continue
        html_item += (
            f"<h4>Annotations ({VLM_MODEL_NAME})</h4>{annotation.text}<br />\n"
        )
    html_buffer.append(html_item)
display.HTML("<hr />".join(html_buffer))

## Resumen

### Lo que has logrado en el Laboratorio 1

¡Felicidades! Has dominado el primer paso crítico en la IA de documentos:

- Conversión de documentos básica y avanzada con retroalimentación visual
- Múltiples formatos de exportación con opciones de visualización
- Extracción de tablas e imágenes con verificación visual
- Modelos de enriquecimiento y VLMs

### Próximos Pasos

Puedes continuar explorando Docling por tu cuenta probando con tus propios documentos o documentos más complejos, o avanzar al siguiente laboratorio en esta serie.

Para más información, explora los recursos a continuación:

- GitHub: https://github.com/docling-project/docling
- Documentation: https://docling-project.github.io/docling/
- Technical Report: https://arxiv.org/abs/2408.09869
- Examples: https://github.com/docling-project/docling/tree/main/examples